In [85]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
#sprawdzenie
print(spark.version)

from pyspark.sql.types import StructType, StructField, IntegerType, StringType

#2

struct_uad = StructType([ 
StructField('UserId', IntegerType(), True),
StructField('ArtistId', IntegerType(), True),
StructField('Plays', IntegerType(), True)])

df_uad = spark.read.format("csv").option("header", "false") \
.option("delimiter", " ") \
.schema(struct_uad) \
.load("user_artist_data.txt")

#3
struct_ad = StructType([ 
StructField('ArtistId', IntegerType(), True),
StructField('ArtistName', StringType(), True)])

df_ad = spark.read.format("csv").option("header", "false") \
.option("delimiter", "\t") \
.schema(struct_ad) \
.load("artist_data.txt")

3.5.1


In [86]:
#4

df_uad.join(df_ad, 'ArtistId').select(df_uad.UserId, df_ad.ArtistName, df_uad.Plays).show(10)

+-------+--------------------+-----+
| UserId|          ArtistName|Plays|
+-------+--------------------+-----+
|1059637|           Aerosmith|  238|
|1059637|     Edna's Goldfish|    1|
|1059637|The Mighty Mighty...|    1|
|1059637|        Foo Fighters|   11|
|1059637|  The Bouncing Souls|    1|
|1059637|       Alkaline Trio|  423|
|1059637|         The Beatles|    5|
|1059637|           Pennywise|    2|
|1059637|             Incubus|    2|
|1059637|         Bright Eyes|19129|
+-------+--------------------+-----+
only showing top 10 rows



In [87]:
#5
print(pow(2,31))
df_uad.summary().show()


2147483648
+-------+------------------+------------------+------------------+
|summary|            UserId|          ArtistId|             Plays|
+-------+------------------+------------------+------------------+
|  count|             49481|             49481|             49481|
|   mean|1328420.1949435137|2003155.0297285826|130.57579677047755|
| stddev| 452991.3131262286|2489609.3644763026|3034.3847545693047|
|    min|           1000647|                 1|                 1|
|    25%|           1024631|           1004015|                 2|
|    50%|           1059245|           1055553|                 7|
|    75%|           2010008|           1313001|                37|
|    max|           2288164|          10788218|            439771|
+-------+------------------+------------------+------------------+



In [88]:
#6
[df_train, df_test] = df_uad.randomSplit([0.8, 0.2])
print(df_test.count())
print(df_train.count())

9929
39552


In [89]:
#7

from pyspark.ml.recommendation import ALS

als = ALS( \
seed=327, \
implicitPrefs=True, \
rank = 10, \
maxIter = 5, \
userCol="UserId", \
itemCol="ArtistId", \
ratingCol="Plays", \
)
model = als.fit(df_train)


In [90]:
#8

model.setColdStartStrategy("drop")
df_predictions= model.transform(df_test)

In [91]:
#9

rec4user10 = model.recommendForAllUsers(10)
rec4user10.show(5,250)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| UserId|                                                                                                                                                                                                   recommendations|
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1001440|[{1001412, 2.733694}, {1028270, 2.4042463}, {1002840, 2.4000485}, {1004226, 2.3963218}, {1259, 2.2687557}, {1004294, 2.1116188}, {1004296, 2.0002396}, {1004484, 1.8663117}, {4496, 1.7934595}, {3111, 1.7176604}]|
|1021940|    [{831, 1.7581882}, {1182660, 1.7393873}, {1003727, 1.7343948}, {234, 1.6247395}, {1233196, 1.6094174}, 

In [92]:
#10

usersId=[{'UserId':1059637},{'UserId':2007381}]
df_uid=spark.createDataFrame(usersId)
df_uid.show()

rec42users5 = model.recommendForUserSubset(df_uid, 5)
rec42users5.show(2,250)


+-------+
| UserId|
+-------+
|1059637|
|2007381|
+-------+

+-------+----------------------------------------------------------------------------------------------------+
| UserId|                                                                                     recommendations|
+-------+----------------------------------------------------------------------------------------------------+
|2007381|[{1007735, 2.613232}, {1000139, 2.3941727}, {352, 2.377395}, {1009156, 2.263288}, {1205, 2.1499245}]|
|1059637|  [{4371, 1.9181359}, {1004226, 1.8840401}, {1, 1.7925165}, {1004484, 1.7697855}, {1854, 1.7184899}]|
+-------+----------------------------------------------------------------------------------------------------+



In [101]:
#11
from pyspark.sql.functions import explode

usersId=[{'UserId':1070932}]
df_uid=spark.createDataFrame(usersId)
df_uid.show()

rec42users1 = model.recommendForUserSubset(df_uid, 10)
rec42users1.show(1,250)

rec = rec42users1.withColumn("rec", explode("recommendations")).select("UserId", "rec.ArtistId", "rec.rating")

rec.show()


+-------+
| UserId|
+-------+
|1070932|
+-------+

+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
| UserId|                                                                                                                                                                                            recommendations|
+-------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1070932|[{1004226, 2.4329066}, {4371, 2.4210553}, {1031866, 2.3644717}, {1004484, 2.2563534}, {1000323, 2.1171982}, {1854, 1.9409239}, {1, 1.82712}, {1044920, 1.6582981}, {1004162, 1.6431652}, {3111, 1.6365258}]|
+-------+------------------------------------------------------------------------------------

In [109]:
#12

from pyspark.sql.functions import desc

joinedList = rec.join(df_ad, 'ArtistId').select(rec.UserId, rec.ArtistId, rec.rating, df_ad.ArtistName)
joinedList2 = joinedList.join(df_uad, 'ArtistId').select(rec.UserId, rec.ArtistId, rec.rating, df_ad.ArtistName, df_uad.Plays)
joinedList2.show()

df_uad.join(df_ad, 'ArtistId') \
.select(df_uad.UserId, df_ad.ArtistName, df_uad.Plays) \
.where(df_uad.UserId == "1070932").orderBy(df_uad.Plays.desc()).show()

AttributeError: 'DataFrame' object has no attribute 'leftJoin'